In [2]:
import gurobipy as gp
from gurobipy import GRB

There are two factories, four distribution centers, and six wholesalers. Each factory has a maximum production capacity, each distribution center has a maximum throughput limit, and each wholesaler requires a minimum amount of shipped product. Costs are associated with shipments to distribution centers and wholesalers.

What is the optimal amount of routing which minimizes the cost of shipping while meeting the requirements of all wholesalers?

In [3]:
# Entities as lists
factories = ['Cleveland', 'Austin']
distCenters = ['Pittsburgh','Boulder','Des Moines','Tucson']
wholesalers = ['w1','w2','w3','w4','w5','w6']

# Data as dictionaries for easier reference later
requirements = {'w1':50000,'w2':10000,'w3':40000,
                'w4':35000,'w5':60000,'w6':20000}
capacities = {'Cleveland':150000,'Austin':200000}
throughputs = {'Pittsburgh':70000,'Boulder':50000,
               'Des Moines':100000, 'Tucson':40000}

# Cost of shipping from each potential origin to each potential destination 
costs = {
    'Cleveland': {
        'Pittsburgh': .5, 'Boulder': .5, 'Des Moines': 1, 'Tucson': .2,
        'w1': 1, 'w2': 0, 'w3': 1.5, 'w4': 2, 'w5': 0, 'w6': 1
    },
    'Austin': {
        'Pittsburgh': 0, 'Boulder': .3, 'Des Moines': .5, 'Tucson': .2,
        'w1': 2, 'w2': 0, 'w3': 0, 'w4': 0, 'w5': 0, 'w6': 0     
    },
    'Pittsburgh': {'w1': 0, 'w2': 1.5, 'w3': .5, 'w4': 1.5, 'w5': 0, 'w6': 1
    },
    'Boulder': {'w1': 1, 'w2': .5, 'w3': .5, 'w4': 1, 'w5': .5, 'w6': 0
    },
    'Des Moines': {'w1': 0, 'w2': 1.5, 'w3': 2, 'w4': 0, 'w5': .5, 'w6': 1.5
    },
    'Tucson': {'w1': 0, 'w2': 0, 'w3': .2, 'w4': 1.5, 'w5': .5, 'w6': 1.5
    }
}

# Model
t = gp.Model("Transportation")
t.ModelSense = GRB.MINIMIZE

# Decision variables
# Factory to Wholesaler
f_w = t.addVars(factories, wholesalers, lb=0.0, name='x')
# Factory to Distribution Center
f_d = t.addVars(factories, distCenters, lb=0.0, name='x') 
# Distribution Center to Wholesaler
d_w = t.addVars(distCenters, wholesalers, lb=0.0, name='x')

# Objective function
# Minimize costs of sending across f_w, f_d, and d_w routes
t.setObjective(gp.quicksum(costs[i][j] * f_w[i,j] for j in wholesalers for i in factories) 
                + gp.quicksum(costs[p][j] * d_w[p,j] for j in wholesalers for p in distCenters)
                + gp.quicksum(costs[i][p] * f_d[i,p] for p in distCenters for i in factories))

# Constraints
# Demand
[t.addConstr(gp.quicksum(f_w[i,j] for i in factories) + gp.quicksum(d_w[p,j] for p in distCenters)
              >= requirements[j], name = f'{wholesalers[n]} demand') for n,j in enumerate(wholesalers)]
# Capacity
[t.addConstr(gp.quicksum(f_w[i,j] for j in wholesalers) + gp.quicksum(f_d[i,p] for p in distCenters) 
              <= capacities[i], name = f'{factories[n]} capacity') for n,i in enumerate(factories)]
# Throughput IN; Route from factory to distribution center cannot exceed throughput
[t.addConstr(gp.quicksum(f_d[i,p] for i in factories) <= throughputs[p],
              name = f'{distCenters[n]} throughput in') for n,p in enumerate(distCenters)]
# Throughput OUT; All incoming product must be shipped from distribution center, and distribution center may not ship more than incoming
[t.addConstr(gp.quicksum(d_w[p,j] for j in wholesalers) == gp.quicksum(f_d[i,p] for i in factories),
              name = f'{distCenters[n]} throughput out') for n,p in enumerate(distCenters)]

# Supply constraints; These routes are unavailable, so no units may be sent along them
# t.addConstr(f_d['Austin','Pittsburgh'] + f_w['Austin','w2'] + f_w['Austin','w3'] + f_w['Austin','w4'] + f_w['Austin','w5'] + f_w['Austin','w6'] 
#              + f_w['Cleveland','w2'] + f_w['Cleveland','w5']
#              + d_w['Pittsburgh','w1'] + d_w['Pittsburgh','w5'] + d_w['Boulder','w6'] + d_w['Des Moines','w1'] + d_w['Des Moines','w4'] 
#              + d_w['Tucson','w1'] + d_w['Tucson','w2'] <= 0, name = "Unavailable routes")

## Running these constraints with individual logic allows me to see the shadow price of each unavailable route.
[t.addConstr(f_d[f,d] <= 0, name = f'{factories[n]}-{distCenters[k]} unavailable') for k,d in enumerate(distCenters) for n,f in enumerate(factories) if costs[f][d] == 0]
[t.addConstr(f_w[f,w] <= 0, name = f'{factories[n]}-{wholesalers[j]} unavailable') for j,w in enumerate(wholesalers) for n,f in enumerate(factories) if costs[f][w] == 0]
[t.addConstr(d_w[d,w] <= 0, name = f'{distCenters[k]}-{wholesalers[j]} unavailable') for j,w in enumerate(wholesalers) for k,d in enumerate(distCenters) if costs[d][w] == 0]
t.update()

# Preference constraints; The model must meet at least these demands, which are coded as at or below the maximum limit for the preferred supplier
## Wholesaler 1
t.addConstr(f_w['Cleveland','w1'] >= 50000, name = "w1 preference")
## Wholesaler 2
t.addConstr(d_w['Pittsburgh','w2'] >=  10000, name = "w2 preference")
## Wholesaler 5; Entered as Boulder's maximum throughput limit
t.addConstr(d_w['Boulder','w5'] >= 50000, name = "w5 preference")
## Wholesaler 6; Either distribution center is acceptable
t.addConstr(d_w['Des Moines','w6'] + d_w['Tucson','w6'] >= 20000, name = "w6 preference")

t.update()
t.display()
t.optimize()

[print(f"{var.varName}: {var.X}") for var in t.getVars() if var.X > 0]

Set parameter Username
Set parameter LicenseID to value 2710302
Academic license - for non-commercial use only - expires 2026-09-18
Minimize
x[Cleveland,w1] + 1.5 x[Cleveland,w3] + 2.0 x[Cleveland,w4] + x[Cleveland,w6]
+ 2.0 x[Austin,w1] + 0.5 x[Cleveland,Pittsburgh] + 0.5 x[Cleveland,Boulder]
+ x[Cleveland,Des Moines] + 0.2 x[Cleveland,Tucson] + 0.3 x[Austin,Boulder]
+ 0.5 x[Austin,Des Moines] + 0.2 x[Austin,Tucson] + 1.5 x[Pittsburgh,w2]
+ 0.5 x[Pittsburgh,w3] + 1.5 x[Pittsburgh,w4] + x[Pittsburgh,w6] + x[Boulder,w1]
+ 0.5 x[Boulder,w2] + 0.5 x[Boulder,w3] + x[Boulder,w4] + 0.5 x[Boulder,w5]
+ 1.5 x[Des Moines,w2] + 2.0 x[Des Moines,w3] + 0.5 x[Des Moines,w5]
+ 1.5 x[Des Moines,w6] + 0.2 x[Tucson,w3] + 1.5 x[Tucson,w4] + 0.5 x[Tucson,w5]
+ 1.5 x[Tucson,w6]
Subject To
w1 demand: x[Cleveland,w1] + x[Austin,w1] + x[Pittsburgh,w1] + x[Boulder,w1] + x[Des
 Moines,w1] + x[Tucson,w1] >= 50000
w2 demand: x[Cleveland,w2] + x[Austin,w2] + x[Pittsburgh,w2] + x[Boulder,w2] + x[Des
 Moines,w2] + 

C:\Users\harri\AppData\Local\Temp\ipykernel_19516\3245848165.py:88: DeprecationWarning: Model.display() is deprecated
  t.display()


[None, None, None, None, None, None, None, None, None, None, None]

In [7]:
# Below is a sensitivity analysis which show where routes may change based on shipping costs.
# The analysis also displays the potential payoff of making expanding route capacity or even making new routes available, derived from the Shadow Price.

for var in t.getVars():
    if var.X > 0:
        print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub}), Reduced cost = {var.RC}, Coeff = {var.obj}, Obj coeff range = ({var.SAObjLow: .3f}, {var.SAObjUp: .3f})')
for c in t.getConstrs():
    if c.Pi < 0:
        print(f'Constraint: {c.ConstrName}, RHS = {c.RHS}, Slack = {c.slack}, RHS = {c.SARHSUp}, Shadow Price = {c.Pi}')

Variable: x[Cleveland,w1], Optimal Value = 50000.0, (LB,UB) = (0.0, inf), Reduced cost = 0.0, Coeff = 1.0, Obj coeff range = ( 0.000,  inf)
Variable: x[Cleveland,Pittsburgh], Optimal Value = 45000.0, (LB,UB) = (0.0, inf), Reduced cost = 0.0, Coeff = 0.5, Obj coeff range = ( 0.500,  0.500)
Variable: x[Austin,Boulder], Optimal Value = 50000.0, (LB,UB) = (0.0, inf), Reduced cost = 0.0, Coeff = 0.3, Obj coeff range = (-inf,  0.500)
Variable: x[Austin,Des Moines], Optimal Value = 30000.0, (LB,UB) = (0.0, inf), Reduced cost = 0.0, Coeff = 0.5, Obj coeff range = ( 0.500,  0.800)
Variable: x[Austin,Tucson], Optimal Value = 40000.0, (LB,UB) = (0.0, inf), Reduced cost = 0.0, Coeff = 0.2, Obj coeff range = (-inf,  0.200)
Variable: x[Pittsburgh,w2], Optimal Value = 10000.0, (LB,UB) = (0.0, inf), Reduced cost = 0.0, Coeff = 1.5, Obj coeff range = (-0.500,  inf)
Variable: x[Pittsburgh,w4], Optimal Value = 35000.0, (LB,UB) = (0.0, inf), Reduced cost = 0.0, Coeff = 1.5, Obj coeff range = ( 1.500,  1.5